# Notebook 02 · Additional Gold Tables: Brand, Category, Price Band

## Purpose

This notebook extends the batch lakehouse built in **Notebook 01** by adding several additional **gold-level aggregate tables**. All of them are derived from the cleaned **silver** events and are designed to support richer analytics and business views:

- Daily funnels by brand  
- Daily funnels by top level category  
- Hourly funnels by price band  

These tables give the business multiple “lenses” on the same event stream and are natural inputs for dashboards, reports, and further anomaly analysis.

---

## Inputs and Dependencies

- **Input table**: `tables/silver_events`  
  - Cleaned, deduplicated event logs across October and November  
  - Normalized brand and category fields  
  - Partitioned by `event_date`

- **Dependencies from Notebook 01**
  - The bronze-to-silver pipeline must have already run successfully
  - `silver_events` must already exist and contain the months you want to analyze

This notebook does **not** read raw CSV files or bronze; it operates purely at the silver and gold layers.

---

## High Level Workflow

1. Initialize Spark and project paths (same environment as Notebook 01)
2. Load the **silver** table
3. Derive useful dimensions:
   - `event_date` (if needed)
   - Top level `category_root`
   - Discrete `price_band` buckets
4. Build three additional gold tables:
   - `gold_funnel_daily_brand`
   - `gold_funnel_daily_category`
   - `gold_funnel_hourly_price_band`
5. Run sanity checks and sample analytics to confirm that metrics and structures are as expected

---

## Dimension Engineering

Before aggregation, the notebook enriches the silver events with additional dimensions that are useful for business analysis:

1. **Event date**
   - Ensures each row has a calendar `event_date` derived from `event_time`
   - Used as the time grain for daily aggregates and as a partition key

2. **Top level category (`category_root`)**
   - Split the normalized `category_code_norm` at the first dot
   - Example: `"beauty.face.cream"` → `"beauty"`
   - Collapses detailed category hierarchies into a manageable top level for executive views

3. **Price band (`price_band`)**
   - Segment products into coarse price ranges, for example:
     - `<10`, `10–30`, `30–60`, `60+`
   - Enables analysis of user behavior and performance across cheap vs premium items

These dimensions are reused across multiple gold tables and can be referenced in dashboards and queries.

---

## Gold Table 1 · `gold_funnel_daily_brand`

**Grain**

- One row per day per brand:
  - `date`  
  - `brand`

**Metrics**

- Aggregated from silver events for each day and brand:
  - `views`      number of view events  
  - `carts`      number of cart events  
  - `purchases`  number of purchase events  
  - `revenue`    sum of purchase prices  
- Derived funnel rates:
  - `view_to_cart_rate`      carts per view  
  - `cart_to_purchase_rate`  purchases per cart  
  - `conversion_rate`        purchases per view  

**Storage**

- Written to `tables/gold_funnel_daily_brand`
- Partitioned by `date` for efficient time-based slicing

**Business usage**

- Tracking daily brand performance over time
- Comparing brands on conversion and revenue trends
- Feeding line charts and leaderboards in dashboards

---

## Gold Table 2 · `gold_funnel_daily_category`

**Grain**

- One row per day per top level category:
  - `date`  
  - `category_root` (e.g., `"beauty"`)

**Metrics**

- Same funnel metrics as for brands, but grouped by category:
  - `views`, `carts`, `purchases`, `revenue`
  - `view_to_cart_rate`, `cart_to_purchase_rate`, `conversion_rate`

**Storage**

- Written to `tables/gold_funnel_daily_category`
- Partitioned by `date`

**Business usage**

- Understanding which major product categories drive traffic and revenue
- Identifying categories with strong conversion vs those needing attention
- Supporting category-level KPI dashboards and long term portfolio planning

---

## Gold Table 3 · `gold_funnel_hourly_price_band`

**Grain**

- One row per hour per price band:
  - 1-hour window on `event_time`  
  - `price_band` (`<10`, `10–30`, `30–60`, `60+`)

**Metrics**

- Same funnel metrics as other tables, but aggregated by price band:
  - `views`, `carts`, `purchases`, `revenue`
  - `view_to_cart_rate`, `cart_to_purchase_rate`, `conversion_rate`

**Storage**

- Written to `tables/gold_funnel_hourly_price_band`
- Partitioned by `window_date` (date extracted from the window start)

**Business usage**

- Analyzing behavior around low, mid, and high priced items
- Seeing how conversion for different price segments changes by time of day
- Supporting pricing strategy and promotional design

---

## Role of This Notebook in the Overall System

This notebook enriches the lakehouse with **multiple gold views** on the same underlying behavioral data:

- Brand lens (daily)
- Category lens (daily)
- Price lens (hourly)

Together with the **hourly brand gold** from Notebook 01, these tables:

- Provide diverse inputs for dashboards and self-serve analytics
- Supply richer context for anomaly investigation (e.g., “Is the drop concentrated in a specific category or price band?”)
- Demonstrate that the system is not limited to a single metric, but can flexibly support multiple aggregated views using the same silver layer

In the full FunnelPulse architecture:

- Notebook 01 builds **bronze → silver → first gold** (hourly brand)
- Notebook 02 adds **additional gold tables** that broaden analytical coverage
- Subsequent notebooks:
  - Construct streaming pipelines that parallel these gold metrics  
  - Build anomaly detection on top of the hourly funnel behavior  

This notebook is therefore the main bridge between the cleaned event layer (silver) and the multi-dimensional business metrics that stakeholders will actually consume.

In [1]:
# CELL 1: Spark init (same style as professor template) and paths

# Show memory and cores
!free -h | grep "Mem:"
!lscpu | grep "CPU(s):"

import os
import pyspark

conf = pyspark.SparkConf()
conf.set('spark.ui.proxyBase', '/user/' + os.environ['JUPYTERHUB_USER'] + '/proxy/4040')
conf.set('spark.sql.repl.eagerEval.enabled', False)
conf.set('spark.driver.memory','6g')

sc = pyspark.SparkContext(conf=conf, master='local[*]')
spark = pyspark.sql.SparkSession.builder.appName("FunnelPulse Additional Gold").getOrCreate()

print(spark)
print(f"Spark UI: https://csgy-6513-fall.rcnyu.org{conf.get('spark.ui.proxyBase')}")

# Project paths
home = os.path.expanduser("~")
project_root = os.path.join(home, "funnelpulse")
tables_dir = os.path.join(project_root, "tables")

silver_path = os.path.join(tables_dir, "silver_events")
gold_daily_brand_path     = os.path.join(tables_dir, "gold_funnel_daily_brand")
gold_daily_category_path  = os.path.join(tables_dir, "gold_funnel_daily_category")
gold_hourly_price_path    = os.path.join(tables_dir, "gold_funnel_hourly_price_band")

print("Silver path              :", silver_path)
print("Gold daily brand path    :", gold_daily_brand_path)
print("Gold daily category path :", gold_daily_category_path)
print("Gold hourly price path   :", gold_hourly_price_path)

Mem:            15Gi       5.6Gi       8.1Gi        40Mi       2.3Gi        10Gi
CPU(s):                                  2
NUMA node0 CPU(s):                       0,1


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/26 11:54:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/11/26 11:54:34 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


Spark UI: https://csgy-6513-fall.rcnyu.org/user/ss18851/proxy/4040
Silver path              : /home/jovyan/funnelpulse/tables/silver_events
Gold daily brand path    : /home/jovyan/funnelpulse/tables/gold_funnel_daily_brand
Gold daily category path : /home/jovyan/funnelpulse/tables/gold_funnel_daily_category
Gold hourly price path   : /home/jovyan/funnelpulse/tables/gold_funnel_hourly_price_band


In [2]:
# CELL 2: read silver and derive helper dimensions

from pyspark.sql.functions import to_date, col, split, when

silver = spark.read.parquet(silver_path)

print("Silver rows:", silver.count())
silver.printSchema()
silver.show(5, truncate=False)

# Ensure we have event_date
if "event_date" not in silver.columns:
    silver = silver.withColumn("event_date", to_date(col("event_time")))

# Derive category_root from category_code_norm
# Example: "beauty.face.cream" -> "beauty"
if "category_code_norm" in silver.columns:
    silver = silver.withColumn(
        "category_root",
        split(col("category_code_norm"), "\\.").getItem(0)
    )
else:
    # safe fallback
    silver = silver.withColumn("category_root", col("category_code"))

# Derive price_band
silver = (
    silver
    .withColumn(
        "price_band",
        when(col("price") < 10, "<10")
        .when((col("price") >= 10) & (col("price") < 30), "10-30")
        .when((col("price") >= 30) & (col("price") < 60), "30-60")
        .otherwise("60_plus")
    )
)

silver.select(
    "event_time", "event_date",
    "brand_norm", "category_code_norm", "category_root",
    "price", "price_band"
).show(10, truncate=False)

Silver rows: 8260755
root
 |-- event_time: timestamp (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- category_id: long (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: double (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- user_session: string (nullable = true)
 |-- brand_norm: string (nullable = true)
 |-- category_code_norm: string (nullable = true)
 |-- dq_missing_session: boolean (nullable = true)
 |-- dq_missing_brand: boolean (nullable = true)
 |-- dq_missing_category: boolean (nullable = true)
 |-- event_date: date (nullable = true)



+-------------------+----------+----------+-------------------+-------------+--------+-----+---------+------------------------------------+----------+------------------+------------------+----------------+-------------------+----------+
|event_time         |event_type|product_id|category_id        |category_code|brand   |price|user_id  |user_session                        |brand_norm|category_code_norm|dq_missing_session|dq_missing_brand|dq_missing_category|event_date|
+-------------------+----------+----------+-------------------+-------------+--------+-----+---------+------------------------------------+----------+------------------+------------------+----------------+-------------------+----------+
|2019-11-22 11:28:38|view      |5734492   |1487580005050352469|NULL         |haruyama|3.76 |228337277|000caab0-80fa-43ef-8abc-327ae98f2358|haruyama  |NULL              |false             |false           |true               |2019-11-22|
|2019-11-22 13:45:57|view      |5751415   |151189274

In [3]:
# CELL 3: gold_funnel_daily_brand (daily funnel metrics by brand)

from pyspark.sql.functions import sum as _sum

# Aggregate by event_date and brand
daily_brand = (
    silver
    .groupBy(
        col("event_date").alias("date"),
        col("brand_norm").alias("brand")
    )
    .agg(
        _sum((col("event_type") == "view").cast("int")).alias("views"),
        _sum((col("event_type") == "cart").cast("int")).alias("carts"),
        _sum((col("event_type") == "purchase").cast("int")).alias("purchases"),
        _sum(
            when(col("event_type") == "purchase", col("price")).otherwise(0.0)
        ).alias("revenue")
    )
)

# Funnel rates
daily_brand = (
    daily_brand
    .withColumn(
        "view_to_cart_rate",
        when(col("views") > 0, col("carts") / col("views"))
    )
    .withColumn(
        "cart_to_purchase_rate",
        when(col("carts") > 0, col("purchases") / col("carts"))
    )
    .withColumn(
        "conversion_rate",
        when(col("views") > 0, col("purchases") / col("views"))
    )
)

daily_brand.printSchema()
daily_brand.orderBy("date", "brand").show(20, truncate=False)

# Write as partitioned table
(
    daily_brand
    .write
    .mode("overwrite")
    .partitionBy("date")
    .parquet(gold_daily_brand_path)
)

print("gold_funnel_daily_brand written to:", gold_daily_brand_path)

root
 |-- date: date (nullable = true)
 |-- brand: string (nullable = true)
 |-- views: long (nullable = true)
 |-- carts: long (nullable = true)
 |-- purchases: long (nullable = true)
 |-- revenue: double (nullable = true)
 |-- view_to_cart_rate: double (nullable = true)
 |-- cart_to_purchase_rate: double (nullable = true)
 |-- conversion_rate: double (nullable = true)



+----------+-----------+-----+-----+---------+------------------+-------------------+---------------------+-------------------+
|date      |brand      |views|carts|purchases|revenue           |view_to_cart_rate  |cart_to_purchase_rate|conversion_rate    |
+----------+-----------+-----+-----+---------+------------------+-------------------+---------------------+-------------------+
|2019-09-30|NULL       |798  |524  |172      |1011.5400000000001|0.656641604010025  |0.3282442748091603   |0.21553884711779447|
|2019-09-30|airnails   |6    |2    |2        |2.38              |0.3333333333333333 |1.0                  |0.3333333333333333 |
|2019-09-30|ardell     |1    |0    |0        |0.0               |0.0                |NULL                 |0.0                |
|2019-09-30|art-visage |9    |7    |3        |8.02              |0.7777777777777778 |0.42857142857142855  |0.3333333333333333 |
|2019-09-30|artex      |1    |0    |0        |0.0               |0.0                |NULL               

[Stage 12:>                                                         (0 + 1) / 1]

gold_funnel_daily_brand written to: /home/jovyan/funnelpulse/tables/gold_funnel_daily_brand


In [4]:
# CELL 4: gold_funnel_daily_category (daily funnel metrics by category_root)

daily_category = (
    silver
    .groupBy(
        col("event_date").alias("date"),
        col("category_root").alias("category_root")
    )
    .agg(
        _sum((col("event_type") == "view").cast("int")).alias("views"),
        _sum((col("event_type") == "cart").cast("int")).alias("carts"),
        _sum((col("event_type") == "purchase").cast("int")).alias("purchases"),
        _sum(
            when(col("event_type") == "purchase", col("price")).otherwise(0.0)
        ).alias("revenue")
    )
)

daily_category = (
    daily_category
    .withColumn(
        "view_to_cart_rate",
        when(col("views") > 0, col("carts") / col("views"))
    )
    .withColumn(
        "cart_to_purchase_rate",
        when(col("carts") > 0, col("purchases") / col("carts"))
    )
    .withColumn(
        "conversion_rate",
        when(col("views") > 0, col("purchases") / col("views"))
    )
)

daily_category.printSchema()
daily_category.orderBy("date", "category_root").show(20, truncate=False)

(
    daily_category
    .write
    .mode("overwrite")
    .partitionBy("date")
    .parquet(gold_daily_category_path)
)

print("gold_funnel_daily_category written to:", gold_daily_category_path)

root
 |-- date: date (nullable = true)
 |-- category_root: string (nullable = true)
 |-- views: long (nullable = true)
 |-- carts: long (nullable = true)
 |-- purchases: long (nullable = true)
 |-- revenue: double (nullable = true)
 |-- view_to_cart_rate: double (nullable = true)
 |-- cart_to_purchase_rate: double (nullable = true)
 |-- conversion_rate: double (nullable = true)



+----------+-------------+-----+-----+---------+------------------+--------------------+---------------------+--------------------+
|date      |category_root|views|carts|purchases|revenue           |view_to_cart_rate   |cart_to_purchase_rate|conversion_rate     |
+----------+-------------+-----+-----+---------+------------------+--------------------+---------------------+--------------------+
|2019-09-30|NULL         |1878 |1455 |522      |2309.9699999999943|0.7747603833865815  |0.35876288659793815  |0.2779552715654952  |
|2019-09-30|accessories  |8    |0    |0        |0.0               |0.0                 |NULL                 |0.0                 |
|2019-09-30|apparel      |3    |3    |3        |24.28             |1.0                 |1.0                  |1.0                 |
|2019-09-30|appliances   |31   |3    |1        |47.46             |0.0967741935483871  |0.3333333333333333   |0.03225806451612903 |
|2019-09-30|furniture    |9    |1    |0        |0.0               |0.1111111

[Stage 18:>                                                         (0 + 1) / 1]

gold_funnel_daily_category written to: /home/jovyan/funnelpulse/tables/gold_funnel_daily_category


In [5]:
# CELL 5: gold_funnel_hourly_price_band (hourly funnels by price band)

from pyspark.sql.functions import window, to_date

hourly_price = (
    silver
    .groupBy(
        window(col("event_time"), "1 hour").alias("w"),
        col("price_band").alias("price_band")
    )
    .agg(
        _sum((col("event_type") == "view").cast("int")).alias("views"),
        _sum((col("event_type") == "cart").cast("int")).alias("carts"),
        _sum((col("event_type") == "purchase").cast("int")).alias("purchases"),
        _sum(
            when(col("event_type") == "purchase", col("price")).otherwise(0.0)
        ).alias("revenue")
    )
)

hourly_price = (
    hourly_price
    .withColumn("window_start", col("w.start"))
    .withColumn("window_end", col("w.end"))
    .drop("w")
)

hourly_price = (
    hourly_price
    .withColumn(
        "view_to_cart_rate",
        when(col("views") > 0, col("carts") / col("views"))
    )
    .withColumn(
        "cart_to_purchase_rate",
        when(col("carts") > 0, col("purchases") / col("carts"))
    )
    .withColumn(
        "conversion_rate",
        when(col("views") > 0, col("purchases") / col("views"))
    )
)

hourly_price = hourly_price.withColumn("window_date", to_date(col("window_start")))

hourly_price.printSchema()
hourly_price.orderBy("window_start", "price_band").show(20, truncate=False)

(
    hourly_price
    .write
    .mode("overwrite")
    .partitionBy("window_date")
    .parquet(gold_hourly_price_path)
)

print("gold_funnel_hourly_price_band written to:", gold_hourly_price_path)

root
 |-- price_band: string (nullable = false)
 |-- views: long (nullable = true)
 |-- carts: long (nullable = true)
 |-- purchases: long (nullable = true)
 |-- revenue: double (nullable = true)
 |-- window_start: timestamp (nullable = true)
 |-- window_end: timestamp (nullable = true)
 |-- view_to_cart_rate: double (nullable = true)
 |-- cart_to_purchase_rate: double (nullable = true)
 |-- conversion_rate: double (nullable = true)
 |-- window_date: date (nullable = true)



+----------+-----+-----+---------+------------------+-------------------+-------------------+-------------------+---------------------+--------------------+-----------+
|price_band|views|carts|purchases|revenue           |window_start       |window_end         |view_to_cart_rate  |cart_to_purchase_rate|conversion_rate     |window_date|
+----------+-----+-----+---------+------------------+-------------------+-------------------+-------------------+---------------------+--------------------+-----------+
|10-30     |114  |48   |10       |162.04            |2019-09-30 20:00:00|2019-09-30 21:00:00|0.42105263157894735|0.20833333333333334  |0.08771929824561403 |2019-09-30 |
|30-60     |15   |3    |1        |47.46             |2019-09-30 20:00:00|2019-09-30 21:00:00|0.2                |0.3333333333333333   |0.06666666666666667 |2019-09-30 |
|60_plus   |16   |2    |0        |0.0               |2019-09-30 20:00:00|2019-09-30 21:00:00|0.125              |0.0                  |0.0                 

[Stage 24:>                                                         (0 + 1) / 1]

gold_funnel_hourly_price_band written to: /home/jovyan/funnelpulse/tables/gold_funnel_hourly_price_band


In [6]:
# CELL 6: sanity checks and simple analytics

gold_daily_brand = spark.read.parquet(gold_daily_brand_path)
gold_daily_category = spark.read.parquet(gold_daily_category_path)
gold_hourly_price = spark.read.parquet(gold_hourly_price_path)

print("Rows in gold_funnel_daily_brand    :", gold_daily_brand.count())
print("Rows in gold_funnel_daily_category :", gold_daily_category.count())
print("Rows in gold_funnel_hourly_price   :", gold_hourly_price.count())

print("\nTop 10 brands by total daily revenue:")
gold_daily_brand.groupBy("brand").agg(
    _sum("revenue").alias("total_revenue")
).orderBy(_sum("revenue").desc()).show(10, truncate=False)

print("\nTop categories by total daily revenue:")
gold_daily_category.groupBy("category_root").agg(
    _sum("revenue").alias("total_revenue")
).orderBy(_sum("revenue").desc()).show(10, truncate=False)

print("\nAverage conversion rate by price band (over all hours):")
gold_hourly_price.groupBy("price_band").agg(
    _sum("purchases").alias("total_purchases"),
    _sum("views").alias("total_views")
).withColumn(
    "overall_conversion_rate",
    when(col("total_views") > 0, col("total_purchases") / col("total_views"))
).orderBy("price_band").show(truncate=False)

Rows in gold_funnel_daily_brand    : 13107
Rows in gold_funnel_daily_category : 374
Rows in gold_funnel_hourly_price   : 5856

Top 10 brands by total daily revenue:


+--------+------------------+
|brand   |total_revenue     |
+--------+------------------+
|NULL    |1094863.5800000005|
|runail  |148254.67999999988|
|grattol |106918.24999999988|
|irisk   |92470.89000000004 |
|uno     |86341.77999999998 |
|strong  |67867.9           |
|masura  |63722.52000000008 |
|jessnail|59633.06999999999 |
|cnd     |59240.76999999999 |
|ingarden|56696.96000000001 |
+--------+------------------+
only showing top 10 rows

Top categories by total daily revenue:
+-------------+------------------+
|category_root|total_revenue     |
+-------------+------------------+
|NULL         |2632126.2600000007|
|appliances   |58724.40000000001 |
|furniture    |27299.89          |
|apparel      |13189.819999999996|
|stationery   |6510.85           |
|accessories  |3901.3899999999994|
|sport        |0.0               |
+-------------+------------------+


Average conversion rate by price band (over all hours):
+----------+---------------+-----------+-----------------------+
|price_